# Write input.md (Choose One Only)

## Comparison

In [1]:
%%writefile input.md
~~~csv
---
table-width: 1.3
---
,pandoc-csv2table,pandoc-placetable,panflute example,my proposal
type,type=simple|multiline|grid|pipe,,,
header,header=yes|no,header=yes|no,header: True|False,header: True|False
caption,caption,caption,title,caption
source,source,file,source,source
aligns,aligns=LRCD,aligns=LRCD,,alignment: LRCD
width,,"widths=""0.5 0.2 0.3""",,"column-width: [0.5, 0.2, 0.3]"
,,inlinemarkdown,,markdown: True|False
,,delimiter,,
,,quotechar,,
,,id (wrapped by div),,
~~~

Overwriting input.md


## Simple Test

%%writefile input.md
~~~csv
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~

## Full Test (This one goes into the docstring)

%%writefile input.md
~~~csv
---
caption: "*Great* Title"
header: False
width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
...
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~


## Testing wrong type

%%writefile input.md
~~~csv
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

---
caption: 0.1
header: IDK
markdown: false
...

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"


---
width:
- -0.1
- -0.2
- -0.3
- -0.4
alignment: 0.1
---
~~~


## Testing Zero Table width

%%writefile input.md
~~~csv
,
,
~~~


## Test external CSV @todo

%%writefile input.md
~~~csv
caption: "*Great* Title"
header: False
width: 0.1, 0.2, 0.3, 0.4
alignment: AlignLeft, AlignRight, AlignCenter, AlignDefault
source: ../source/grid.csv
~~~

# Write Filter

In [2]:
%%writefile ../bin/csv-tables.py
#!/usr/bin/env python3

"""
Panflute filter to parse CSV in fenced YAML code blocks

5 metadata keys are recognized:

-   caption: the caption of the table. If omitted, no caption will be inserted.
-   header: If true, has a header row. default: true
-   width: a list of relative width corresponding to the width of each columns.
    default: auto calculate from the length of line in a (potentially multiline) cell.
-   table-width: the relative width of the table (comparing to, say, \linewidth).
    default: 1.0
-   alignment: a string of characters among L,R,C,D, case-insensitive,
    corresponds to Left-aligned, Right-aligned, Center-aligned, Default-aligned respectively.
    e.g. LCRD for a table with 4 columns
    default: DDD...
-   markdown: If CSV table cell contains markdown syntax or not. default: True

When the metadata keys is invalid, the default will be used instead.

e.g.

```markdown
~~~csv
caption: "*Great* Title"
header: False
width:
  - 0.1
  - 0.2
  - 0.3
  - 0.4
alignment: LRC
---
**_Fruit_**,~~Price~~,_Number_,`Advantages`
*Bananas~1~*,$1.34,12~units~,"Benefits of eating bananas 
(**Note the appropriately
rendered block markdown**):    

- _built-in wrapper_        
- ~~**bright color**~~

"
*Oranges~2~*,$2.10,5^10^~units~,"Benefits of eating oranges:

- **cures** scurvy
- `tasty`"
~~~
```
"""

import io
import csv
import panflute

def get_table_options(options):
    """
    It parses the options output from `panflute.yaml_filter` and
    return it as variables `(caption, width, table_width, alignment, header, markdown)`.
    
    It also check if the followings are valid:
    
    - `width` set to `None` when invalid, each element in `width` set to `0` when negative
    - `table_width`: set to `1.0` if invalid or not positive
    - set `header` to `True` if invalid
    - set `markdown` to `True` if invalid
    
    `caption` is assumed to contain markdown, as in standard pandoc YAML metadata
    
    """
    # get caption
    caption = options.get('caption')
    # get width
    try:
        width = options.get('width')
        width = [(float(x) if float(x) >= 0 else 0) for x in width]
    except (TypeError, ValueError):
        width = None
    # get table_width
    try:
        table_width = options.get('table-width',1.0)
        table_width = float(table_width) if float(table_width) > 0 else 1.0
    except (TypeError, ValueError):
        table_width = 1.0
    # get alignment
    alignment = options.get('alignment')
    # get header
    header = options.get('header',True)
    if not isinstance(header, bool):
        if str(header).lower() == "false":
            header = False
        else:
            header = True
    # get markdown
    markdown = options.get('markdown',True)
    if not isinstance(markdown, bool):
        if str(markdown).lower() == "false":
            markdown = False
        else:
            markdown = True
    return (caption, width, table_width, alignment, header, markdown)

def parse_csv(data, markdown):
    # read csv and convert to panflute table representation
    with io.StringIO(data) as f:
        raw_table_list = list(csv.reader(f))
    body = []
    for row in raw_table_list:
        if markdown:
            cells = [panflute.TableCell(*panflute.convert_text(x)) for x in row]
        else:
            cells = [panflute.TableCell(panflute.Plain(panflute.Str(x))) for x in row]
        body.append(panflute.TableRow(*cells))
    # get no of columns of the table
    number_of_columns = len(raw_table_list[0])
    return (body, number_of_columns, raw_table_list)

def parse_metadata(width, table_width, caption, alignment, number_of_columns, raw_table_list):
    # transform metadata
    ## calculate width
    if width == None:
        width_abs = [max([max([len(line) for line in row[i].split("\n")]) for row in raw_table_list]) for i in range(number_of_columns)]
        width_tot = sum(width_abs)
        try:
            width = [width_abs[i]/width_tot*table_width for i in range(number_of_columns)]
        except ZeroDivisionError:
            width = None
    ## convert caption from markdown
    if caption != None:
        caption = panflute.convert_text(str(caption))[0].content
    ## convert alignment string into pandoc format (AlignDefault, etc.)
    if alignment != None:
        alignment = str(alignment)
        parsed_alignment = []
        for i in range(number_of_columns):
            try:
                if alignment[i].lower() == "l":
                    parsed_alignment.append("AlignLeft")
                elif alignment[i].lower() == "c":
                    parsed_alignment.append("AlignCenter")
                elif alignment[i].lower() == "r":
                    parsed_alignment.append("AlignRight")
                else:
                    parsed_alignment.append("AlignDefault")
            except IndexError:
                for i in range(number_of_columns-len(parsed_alignment)):
                    parsed_alignment.append("AlignDefault")
        alignment = parsed_alignment
    return (width, caption, alignment)

def csv2table(options, data, element, doc):
    # read YAML metadata
    caption, width, table_width, alignment, header, markdown = get_table_options(options)
    body, number_of_columns, raw_table_list = parse_csv(data, markdown)
    width, caption, alignment = parse_metadata(width, table_width, caption, alignment, number_of_columns, raw_table_list)
    # finalize table according to metadata
    header_row = body.pop(0) if header else None # panflute.TableRow(*[panflute.TableCell() for i in range(number_of_columns)]) # for panflute < 1.4.3
    table = panflute.Table(*body, header=header_row, caption=caption, width=width, alignment=alignment)
    return table

# We'll only run this for CodeBlock elements of class 'csv'
def main(doc=None):
     return panflute.run_filter(panflute.yaml_filter, tag='csv', function=csv2table, strict_yaml=True)

if __name__ == '__main__':
    main()

Overwriting ../bin/csv-tables.py


In [3]:
!chmod +x ../bin/csv-tables.py

# Output

## Direct output

In [4]:
!pandoc input.md -t json | ../bin/csv-tables.py

{"pandoc-api-version":[1,17,0,4],"meta":{},"blocks":[{"t":"Table","c":[[],[{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"},{"t":"AlignDefault"}],[0.08666666666666667,0.3838095238095238,0.24761904761904763,0.22285714285714286,0.35904761904761906],[[],[{"t":"Para","c":[{"t":"Str","c":"pandoc-csv2table"}]}],[{"t":"Para","c":[{"t":"Str","c":"pandoc-placetable"}]}],[{"t":"Para","c":[{"t":"Str","c":"panflute"},{"t":"Space"},{"t":"Str","c":"example"}]}],[{"t":"Para","c":[{"t":"Str","c":"my"},{"t":"Space"},{"t":"Str","c":"proposal"}]}]],[[[{"t":"Para","c":[{"t":"Str","c":"type"}]}],[{"t":"Para","c":[{"t":"Str","c":"type=simple|multiline|grid|pipe"}]}],[],[],[]],[[{"t":"Para","c":[{"t":"Str","c":"header"}]}],[{"t":"Para","c":[{"t":"Str","c":"header=yes|no"}]}],[{"t":"Para","c":[{"t":"Str","c":"header=yes|no"}]}],[{"t":"Para","c":[{"t":"Str","c":"header:"},{"t":"Space"},{"t":"Str","c":"True|False"}]}],[{"t":"Para","c":[{"t":"Str","c":"header:"},{"t":"Space"},{

## Native Output

In [5]:
!pandoc --filter=../bin/csv-tables.py input.md -t native

[Table [] [AlignDefault,AlignDefault,AlignDefault,AlignDefault,AlignDefault] [8.666666666666667e-2,0.3838095238095238,0.24761904761904763,0.22285714285714286,0.35904761904761906]
 [[]
 ,[Para [Str "pandoc-csv2table"]]
 ,[Para [Str "pandoc-placetable"]]
 ,[Para [Str "panflute",Space,Str "example"]]
 ,[Para [Str "my",Space,Str "proposal"]]]
 [[[Para [Str "type"]]
  ,[Para [Str "type=simple|multiline|grid|pipe"]]
  ,[]
  ,[]
  ,[]]
 ,[[Para [Str "header"]]
  ,[Para [Str "header=yes|no"]]
  ,[Para [Str "header=yes|no"]]
  ,[Para [Str "header:",Space,Str "True|False"]]
  ,[Para [Str "header:",Space,Str "True|False"]]]
 ,[[Para [Str "caption"]]
  ,[Para [Str "caption"]]
  ,[Para [Str "caption"]]
  ,[Para [Str "title"]]
  ,[Para [Str "caption"]]]
 ,[[Para [Str "source"]]
  ,[Para [Str "source"]]
  ,[Para [Str "file"]]
  ,[Para [Str "source"]]
  ,[Para [Str "source"]]]
 ,[[Para [Str "aligns"]]
  ,[Para [Str "aligns=LRCD"]]
  ,[Para [Str "aligns=LRCD"]]
  ,[]
  ,[Para [Str "alignment:",Space,St

## Markdown Output

In [6]:
!pandoc --filter=../bin/csv-tables.py input.md -t markdown

+--------+-----------------------------+-------------------+------------------+---------------------------+
|        | pandoc-csv2table            | pandoc-placetable | panflute example | my proposal               |
+========+=============================+===================+==================+===========================+
| type   | type=simple|multiline|grid| |                   |                  |                           |
|        | pipe                        |                   |                  |                           |
+--------+-----------------------------+-------------------+------------------+---------------------------+
| header | header=yes|no               | header=yes|no     | header:          | header: True|False        |
|        |                             |                   | True|False       |                           |
+--------+-----------------------------+-------------------+------------------+---------------------------+
| captio | caption          